In [8]:
import os
import numpy as np

In [4]:
def write_to_file(contents, path):
    with open(path, 'w') as f:
        f.write(contents)
    
def read_from_file(path):
    with open(path) as f:
        return ''.join(f.readlines())
    
def find_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind,ind+sll-1
    return -9999, -9999

In [2]:
codes_path = [f for f in os.listdir('prompts_folder/new_function_dataset_builder/codes_simplified') if f.endswith('sol')]
len(codes_path)

25

In [3]:
codes_path = [f for f in os.listdir('prompts_folder/new_function_dataset_builder/codes') if f.endswith('sol')]
len(codes_path)

40

In [6]:
lengths = []
for p in codes_path:
    p = 'prompts_folder/new_function_dataset_builder/codes/' + p
    lengths.append(len(read_from_file(p).split('\n')))

In [9]:
np.average(lengths)

304.575

In [ ]:
\section{Evaluation}
% We select DAO that use full proposal when submit to the governance contract, we select . 3161 proposal from DAOstack, 2310 proposals from DAOhaus, 1746 proposals from Compound\_based, 15443 proposals from Aragon.
In the evaluation section, we aim to answer the following three research questions.

To begin with, we want to evaluate the effectiveness of our proposed framework as a whole. This can be done from two aspects. First, we can measure the effectiveness by comparing the end results (the final codes after all rounds of updates) of our method to the implementations of the functions in the original contract codes; this shows how close to the groundtruth codes our generated codes are. Second, we compare our final codes to the initial codes generated by a simple GPT-4 agent that did not involve any enhancements yet; this shows the continuous improvements and corrections that our framework injected to the initially generated codes, which distinguishes our result from the straightforward generation capabilities of GPT-4.. 
Overall, the first research question is summarized in the following as RQ1:

\begin{tcolorbox}[size=title]
\textbf{RQ1:} How does our feedback system improve the quality of generation, compared to traditional ChatGPT-based approaches?
\end{tcolorbox}

\jinan{Some descriptions of RQ2 goes here}

\begin{tcolorbox}[size=title]
\textbf{RQ2:} What is the performance of each type of agent? How does each types of agent affect the evolution and correction of the generated codes? 
%\todo{compare the outputs of each agent, and see how the codes are improved round by round, and see which agent contributed to the most improvement}

%\todo{contribution: by examining the effect of each agent individually, we get to reveal the effect of each type of way that attemps to improve the quality of code generation by GPT}
\end{tcolorbox}


\jinan{Some descriptions of RQ3 goes here}
\begin{tcolorbox}[size=title]
\textbf{RQ3:} How effective is our gas optimization system? 
%\todo{e.g. construct a small dataset to test how much of the gas inefficient patterns it could find, as well as examples from the original papers. Also, could report how many gas inefficient patterns are found in the generated codes, which reveals the fundamental capabilities of GPT to find gas inefficient patterns}
\end{tcolorbox}



\subsection{Code Generation Task Dataset}
In this section, we describe the steps that we took to establish a new dataset called \textbf{SolGenEval} (\textbf{S}olidity \textbf{G}eneration \textbf{E}valuation) that aims to evaluate the code generation framework that we proposed. Overall, the major steps that we took to construct the dataset are listed below.


\noindent\textbf{STEP 1: Candidate Contract Selecting} 

We aim to select candidate contracts from the most frequently called contracts, since a high frequency of usage is a good indicator of code quality as well as range of applicability. To accomplish this, we extracted the top 5 contract addresses called by contracts that implemented the Compound Protocol ecosystem. This ecosystem has the benefit of having covered a very wide range of language features and use cases of Solidity and having been widely adopted in popular contracts. 

\textbf{STEP 2: Contract Codes Extending}

At this step, we extract the codes of the target deployed contract addresses. Each deployed contract address could contain multiple code files with different contracts performing various functionalities.
Overall, we ended up with 40 code files with 304.575 lines of code on average. 

Note that we did not just take the codes files that contain the target functions that were more frequently called, and rather took all the contract files under the ecosystem of each of the target addresses. 
The main motivation behind this design choice is that contracts codes obtained from STEP 1 are standalone files from individual projects, which lacks comprehensiveness and completeness. To address that, the added contexual contracts provides code files that constitutes a richer coding dataset.
The intuition here is that we want to test the ability of our framework to make generations of various types of functionalities within a Solidity software project. 


\todo{(argument of why do this: we manually examined other contracts and found that )}
For the following reasons:
1. Due to the rate limiting features of the GPT-4 API, it is very difficult to carry out large scale automatic testing that cover a lot of contracts and API calls.
2. The selected top 5 contracts, after extending, already covers a lot of different contracts and comprehensive language features of Solidity \todo{(e.g. discuss how much of the language features are covered, maybe refer to https://solidity-by-example.org/). also make sure to discuss the wide range of use cases}. 



\textbf{STEP 3: contract refactoring}
At this step, we perform refactoring on these contracts, with the goal of underscoring one function of a given contract at a time. Specifically, this is done by 
\todo{maybe put an algorithm section here}

The motivation behind this design choice is that we want to test the capability of our framework to complete functions in a given software project. 

\todo{fill in here}
This step results in .... refactored function code files.


Refactoring the contracts into contracts with blanks, with one blank function each.
Wrote a description for each missing function

The intuition is to test the performance of our framework in the use scenario of completing a missing function, given contexts.

\todo{put a picture comparison of examples from the two datasets, from remix}

\todo{considering if we should do this or not}
Note that we make the following optimizations/considerations during this step:
1. removed functions that mainly rely on directing function calls to another (proxy functions)
2. removed functions that are too short, meaningless, or too simple (typically less than 3 lines) (we want our dataset to be more challenging as GPT-4 is already capable to perform generations on very basic functions, and we target more complex ones.)

also removed comments that preceed the function so this does not constitute extra information for the LLM.

We ended up with 626 refactored contracts, where each contract contains one function.


\textbf{STEP 4: Simplification and Deduplication}

It is worth noting that we have obtained a large number of codes in the previous step, and for the following reasons (copy from the previous section), we want to reduce the number of files that we are to deal with, and to put the spotlight in the most note-worthing code files.


1. took the top 5
2. deduplication
3. removed codes that are too short


We perform deduplication at this step.

\todo{note that we also removed functions that have the same function name, even if they might not have the same signature (i.e. parameters), this is because we want to:
1. maximally diversify our dataset, as having the same function name but different parameter list still could mean that they have more similar implementations than they would for those that have different names. In other words, we want to use this to maximally avoid the case of our datasets being biased toward a certain type of function }

The single-function contracts obtained from the previous step could contain functions that have duplicate names. This is common because multiple different contracts could be employing a similar or same funcionality as one of the intermediate steps, and this could be feflected by calling a function that have the same name.

To reduce the redundancy of our dataset, we decided to only keep only one of the refactored one-function contracts that are duplicates. The filtering is done by random selection. This has the following benefits to our dataset:
1. more diversified
2. less bias and more well balanced (since otherwise the evaluation statistics might be a little skewed towards reflecting single-function contracts that have a higher presence)

\todo{For example, the function ??? appeared ??? times in the original dataset, where this unbalanced presence could skew the testing results}

After this round of deduplication, we ended up with 291 single-function contracts, where each contract contains one function.


\textbf{STEP 5: }
We created two datasets, each containing a different version of each file, called \textbf{\textit{D\textsubscript{p}}} and \textbf{\textit{D\textsubscript{r}}}

\todo{consider making it a curly d}

1. with the function present
2. with the function removed

This 

\textbf{STEP 6: Code File summarization}

%\todo{we try to make our summary at the first step as brief as possible, while preseving the essential information of the codes, with the consideration that we want the most of the generation work to be genuinely done by GPT.}

\todo{contribution: we created a carefully curated dataset which contains code files that cover a wide range of common use cases as well as language features of Solidity. With this, we also attached a set of code summaries for a selected subset of the more complex and frequently used code files from the repository. This has the hope of testing our framework under the generation of more complex cases, since we assume that (cite white paper) GPT is already to make reasonable generations }

We summarized important information from each file, including the input format, output format, and a summary of the purpose of the codes. 

An example is given below: \todo{put a picture here}

The high level idea of the motivation behind this design choice is that 


\noindent\textbf{Discussion:}

\todo{before making SolgenEval public, need to check format issues}

1. the SolgenEval could be easily extended following the same steps, and it is not difficult to extend the size of data to the order of tens of thoudsands.

2. (Inspired by the "Evaluating Large Language Models Trained on Code" paper)

3. generalizability:
1. The Compound Protocol is a popular protocol that are implemented by many Solidity projects
2. The contracts and functionalities that we selected contain rich semantic and syntactic features of Solidity, including modifiers, inheritance, 
it also contains many features and use cases that are widely seen in Decentralized Applications (DApps). 
3. also rich use cases


Overall, here are some statistics about the dataset that we have constructed, recorded at Table~\ref{tab:dataset_stats}:


\begin{table}
\centering
\footnotesize
\caption{\small{Statistics of the dataset}}
\vspace{-2mm}
\begin{tabular}{lc} 
\toprule
Type        & Number   \\
\hline \hline 
Number of target contract addresses        &              \\
Number of contracts          &               \\
Number of extended code files           &               \\ 
Average number of lines of each code file          &               \\
\todo{Put some more rows here}          &               \\


\hline 
todo               &               \\
\bottomrule
\end{tabular}
\label{tab:dataset_stats}
\vspace{-2mm}
\end{table}




 

\subsection{Comparing to GPT-4 (RQ1)}
% \noindent \textbf{Motivation.} The efficiency of the sentence classification module plays an important role in extracting the description facts from the code behavior related sentences. We try to evaluate the precision and recall rate of our sentence classification module. 
\sectionWritingPlan{JJJ}

\noindent \textbf{Approach.} 

% What types of API calls are needed and their timeline

core idea: compare the final codes to the original ones by manual verification

The authors of this paper read the modifications made to the original codes, examined each of the changes, and made corrections to inappropriate or incorrect lines of change where necessary. 

How to evaluate the generated codes?
1. By manual reading, and report the percentage of codes that have:
Incorrect, incomplete, and imprecise generated functions
2. By running the compiler as well as some defect-finding tools, report the average number of issues in the generated codes
3. Check how the gas optimization works (note: need to make sure that the gas optimization does not )
Report the average number of gas issues per document
Report the running gas before and after

we could also present similar figures to the RQ2

we could also report a categorization of the modifications that are made to the codes.



\noindent \textbf{Result.} 


\noindent \textbf{Answer to RQ1.} 

\todo{put some preliminary discussion here as well}
\begin{tcolorbox}[
    standard jigsaw,
    opacityback=0,  % this works only in combination with the key "standard jigsaw"
]
 Finding 1: After a round of careful reading and analysis, we find that ... (describe how the final codes are similar and dissimilar to the original ones. If dissimilar, explain any potential reasons)
\end{tcolorbox}
\vspace{-1mm}


\subsection{Effectiveness of each phase of the framework (RQ2)}

\noindent \textbf{Approach.} 
We perform a two-fold evaluation on the intermediate outputs of each agent.

First, we examine the intermediate output of each agent and report statistics about the effects of each step of our proposed framework.

Second, we compare the intermediate results of each agent and see how the codes get evolved over each step. For example, one of our evaluations attempts to figure out how codes changed, at the level of number of added and deleted codes, from ${CODE_{DirGen}}$ to ${CODE_{CoT}}$, from ${CODE_{DirGen}}$ to ${CODE_{FSE}}$, from ${CODE_{CoT}}$ to ${CODE_{FSE}}$, and so on.

\noindent \textbf{Result.} 

\textbf{PART 1. The Chain of Thought (CoT) Feedback Phase: } the number of steps in the step-by-step CoT plan phase are plotted in the CDF graph in Figure~\ref{fig:cdf_num_steps}. 

The Cumulative Distribution Function (CDF) plot in Figure~\ref{fig:cdf_num_steps} presents the distribution of the number of steps across multiple step-by-step plans. The x-axis denotes the number of steps, while the y-axis represents the cumulative probability. The monotonic increase of the curve from 0 to 1 reveals that as the number of steps in a plan increases, so does the cumulative probability. The steepness of the curve signifies the frequency of plans with a specific number of steps, with a steeper curve indicating a higher probability. The curve's approach to 1 on the far right suggests an upper limit on the number of steps in the majority of plans. The CDF plot, therefore, offers valuable insights into the complexity pattern of our planning system.

From Figure~\ref{fig:RQ2_STEP1_COT_CDF_num_steps} as well as the associated data, we can read that less than 64\% of the plans that were proposed by the planning agent \todo{agent name}. In addition, the minimum number of steps is 4 and the maximum is 20, which exhibits that the proposed plan has varied complexity.
\todo{we could also show a comparison to the lines of codes of the original codes (hUman written)}
In addition, Figure~\ref{fig:RQ2_STEP1_COT_BAR_num_steps} also shows the bar graph distribution of the number of steps proposed at each step. 

\begin{figure}[t]
	\centering
	\includegraphics[width=0.5\textwidth]{figures/RQ2_STEP1_COT_CDF_num_steps.pdf}
	\vspace{-2ex}
	\caption{\small{The percentage of codes that have each specific number of steps}}
       \vspace{-4ex}
	\label{fig:RQ2_STEP1_COT_CDF_num_steps}
\end{figure}

\begin{figure}[t]
	\centering
	\includegraphics[width=0.5\textwidth]{figures/RQ2_STEP1_COT_BAR_num_steps.pdf}
	\vspace{-2ex}
	\caption{\small{No. of steps in the proposed step-by-step plan}}
       \vspace{-4ex}
	\label{fig:RQ2_STEP1_COT_BAR_num_steps}
\end{figure}

\textbf{PART 2. The Few-Shot Example (FSE) Feedback Phase: } for this part, we count the number of modification suggestions that are proposed by the GPT-4 agents for each code file during the FSE phase. The results are plotted in the CDF graph in Figure~\ref{fig:RQ2_STEP2_FSE_CDF_num_mods} as well as the bar graph in Figure~\ref{fig:RQ2_STEP2_FSE_BAR_num_mods}, which follows the same format as in PART 1. 

\jinan{some discussion of the figures goes here}

\begin{figure}[t]
	\centering
	\includegraphics[width=0.5\textwidth]{figures/RQ2_STEP2_FSE_BAR_num_mods.pdf}
	\vspace{-2ex}
	\caption{\small{No. of modifications made to the codes}}
       \vspace{-4ex}
	\label{fig:RQ2_STEP2_FSE_BAR_num_mods}
\end{figure}

\begin{figure}[t]
	\centering
	\includegraphics[width=0.5\textwidth]{figures/RQ2_STEP2_FSE_CDF_num_mods.pdf}
	\vspace{-2ex}
	\caption{\small{The percentage of codes that have each specific number of modifications}}
       \vspace{-4ex}
	\label{fig:RQ2_STEP2_FSE_CDF_num_mods}
\end{figure}

\textbf{PART 3. Related Functions (RF) Feedback: } for this part, we count the number of modification suggestions that are proposed by the GPT-4 agents for each code file during the RF phase. The results are plotted in the CDF graph in Figure~\ref{fig:RQ2_STEP3_RE_CDF_num_mods} as well as the bar graph in Figure~\ref{fig:RQ2_STEP3_RE_BAR_num_mods}, which follows the same format as in PART 1. 

\jinan{some discussion of the figures goes here}


\begin{figure}[t]
	\centering
	\includegraphics[width=0.5\textwidth]{figures/RQ2_STEP3_RE_BAR_num_mods.pdf}
	\vspace{-2ex}
	\caption{\small{No. of modifications made to the codes}}
       \vspace{-4ex}
	\label{fig:RQ2_STEP3_RE_BAR_num_mods}
\end{figure}

\begin{figure}[t]
	\centering
	\includegraphics[width=0.5\textwidth]{figures/RQ2_STEP3_RE_CDF_num_mods.pdf}
	\vspace{-2ex}
	\caption{\small{The percentage of codes that have each specific number of modifications}}
       \vspace{-4ex}
	\label{fig:RQ2_STEP3_RE_CDF_num_mods}
\end{figure}

Table~\ref{tab:num_modified_lines}




\begin{table}
\centering
\footnotesize
\caption{\small{The average number of added, deleted, and total number of modified lines for each code file, for each combination of steps}}
\vspace{-2mm}
\begin{tabular}{c|ccc} 
\toprule
\textbf{Step Sequence}        & \textbf{\# added lines}  & \textbf{\# deleted lines} & \textbf{\# modified lines}   \\
\hline \hline 
STEP 1 $\rightarrow$ STEP 2         & 0.8679   & 0.5283 & 1.3962          \\
STEP 1 $\rightarrow$ STEP 3       & 3.5472      & 2.3774      & 5.9245                   \\
STEP 1 $\rightarrow$ STEP 4      &4.3774       &3.1698       & 7.5477                    \\
STEP 2 $\rightarrow$ STEP 3      & 3.000      & 2.1887      & 5.1887                    \\ 
STEP 2 $\rightarrow$ STEP 4      &3.9057       &3.0377       &6.9434                     \\
STEP 3 $\rightarrow$ STEP 4      &1.7170       &1.6415       &3.3585                     \\

\hline 
\textbf{Average Number}  & \textbf{2.9025}    & \textbf{2.1572}    & \textbf{5.0597}             \\
\bottomrule
\end{tabular}
\label{tab:num_modified_lines}
\vspace{-2mm}
\end{table}




\begin{figure*}[htp]
\centering
\begin{subfigure}[h]{0.3\textwidth}
    \centering
    \includegraphics[width=\textwidth]{figures/RQ2_STEP1_TO_STEP_2.pdf}
    \vspace{-2mm}
    \caption{\small{DG to CoT}}
    \label{subfig:RQ2_STEP1_TO_STEP_2}
\end{subfigure}\quad
\begin{subfigure}[h]{0.3\textwidth}
	\centering
	\includegraphics[width=\textwidth]{figures/RQ2_STEP1_TO_STEP_3.pdf}
	\vspace{-2mm}
    \caption{\small{DG to FSE}}
    \label{subfig:RQ2_STEP1_TO_STEP_3}
\end{subfigure}\quad
\begin{subfigure}[h]{0.3\textwidth}
	\centering
	\includegraphics[width=\textwidth]{figures/RQ2_STEP1_TO_STEP_4.pdf}
	\vspace{-2mm}
    \caption{\small{DG to RF}}
    \label{subfig:RQ2_STEP1_TO_STEP_4}
\end{subfigure}
\vspace{-4mm}
\caption{\small{The statistics of modified content. }}
\label{fig:NOT_NEEDED_1}
\end{figure*}

\begin{figure*}[htp]
\centering
\begin{subfigure}[h]{0.3\textwidth}
    \centering
    \includegraphics[width=\textwidth]{figures/RQ2_STEP2_TO_STEP_3.pdf}
    \vspace{-2mm}
    \caption{\small{CoT to FSE}}
    \label{subfig:RQ2_STEP2_TO_STEP_3}
\end{subfigure}\quad
\begin{subfigure}[h]{0.3\textwidth}
	\centering
	\includegraphics[width=\textwidth]{figures/RQ2_STEP2_TO_STEP_4.pdf}
	\vspace{-2mm}
    \caption{\small{CoT to RF}}
    \label{subfig:RQ2_STEP2_TO_STEP_4}
\end{subfigure}\quad
\begin{subfigure}[h]{0.3\textwidth}
	\centering
	\includegraphics[width=\textwidth]{figures/RQ2_STEP3_TO_STEP_4.pdf}
	\vspace{-2mm}
    \caption{\small{FSE to RF}}
    \label{subfig:RQ2_STEP3_TO_STEP_4}
\end{subfigure}
\vspace{-4mm}
\caption{\small{The statistics of modified content. }}
\label{fig:NOT_NEEDED_2}
\end{figure*}



\noindent \textbf{Answer to RQ2.} 

\begin{tcolorbox}[
    standard jigsaw,
    opacityback=0,  % this works only in combination with the key "standard jigsaw"
]
 Finding 3: Overall, our DG -> CoT -> FSE -> RF code updating sequence continuously performs improvements on the initially generated codes.
\end{tcolorbox}
\vspace{-1mm}



\subsection{Gas optimization effectiveness (RQ3)}

 % some statistics that we could report:
% 1. the number of gas inefficient cases that are found
% 2. the amount of gas that is reduced after running and implementing the gas reduction feedback

\noindent \textbf{Approach.} 
To answer the RQ3, we build a set of datasets aiming to examine the effectiveness of the GPT-model to detect gas inefficient patterns.

Note: report the number of each type of gas inefficiency in the dataset, and the percentage of discovery of each type.

We also use other language models as a comparison (e.g. Claud-Instant, Palm, ChatGPT-3.5), to show how well GPT-4 performs in terms of finding gas inefficient patterns following the templates.
(this constitutes a table)

In addition, for each report of gas-inefficient case, manually verify it and report the number of false/true cases.


%\todo{Use GPT to generate test cases}


\noindent \textbf{Result.} 

The result is shown in Table~\ref{tab:gas_opt_accuracy}, which shows that the gas optimization is capable of perform detection of inefficient gas patterns at a high accuracy.
 
\begin{table}
\centering  
\footnotesize
\caption{\small{Detection accuracy of the gas optimization agent}}
\vspace{-2mm}
\begin{tabular}{cccc}  
\toprule
\textbf{Pattern type}        & \textbf{Detected}   \\
\hline \hline 
1. dead code         & \checkmark              \\
2. Opaque predicate          & \checkmark              \\
3. Expensive operations in a loop            & \checkmark               \\ 
4. Constant outcome of a loop          & \checkmark              \\
5. Loop fusion         & \checkmark              \\
6. Repeated computations in a loop         & \checkmark              \\
7. Comparison with unilateral outcome in a loop          & \checkmark              \\

\hline 
\textbf{Percentage of patterns detected}               & \textbf{100\%}              \\
\bottomrule
\end{tabular}
\label{tab:gas_opt_accuracy}
\vspace{-2mm}
\end{table}


\noindent \textbf{Answer to RQ3.} 

\begin{tcolorbox}[
    standard jigsaw,
    opacityback=0,  % this works only in combination with the key "standard jigsaw"
]
 Finding 4: We find that GPT-4 is capable to match against prescribed patterns at a very high accuracy. Specifically we were able to find ... (add stuff about the findings here)
\end{tcolorbox}
\vspace{-1mm}


\todo{need to modify this}
The gas optimization is capable of perform detection of inefficient gas patterns at a high accuracy.












% No.1 0x3d9819210a31b4961b30ef54be2aed79b9c9cd3b
% ABI for the implementation contract at 0xbafe01ff935c7305907c33bf824352ee5979b526, likely using a custom proxy implementation.
% Previously recorded to be on 0x234b619b4f4e405665f7d94f2ce60c24256032b5.
% Contract name: Unitroller
% Contract at 0x3d9819210a31b4961b30ef54be2aed79b9c9cd3b
% Proxy name: Comptroller
% Proxy at: 0xbafe01ff935c7305907c33bf824352ee5979b526


% No.2 '0x0831172b9b136813b0b35e7cc898b1398bb4d7e7'
%  ABI for the implementation contract at 0xc6df585f8721bfafbb1580bd4034315696eab9ca, using the EIP-1967 Transparent Proxy pattern.
% Previously recorded to be on 0x8e516a4357d217405313e40b18357b82bb460917.
% Contract name: ERC1967Proxy
% Contract at '0x0831172b9b136813b0b35e7cc898b1398bb4d7e7'
% Proxy name: GovernorBravoDelegate
% Proxy at: 0xc6df585f8721BfafBb1580bD4034315696EAB9cA


% No.3 0x4dCf7407AE5C07f8681e1659f626E114A7667339
% ABI for the implementation contract at 0xc6df585f8721bfafbb1580bd4034315696eab9ca, using the EIP-1967 Transparent Proxy pattern.
% Previously recorded to be on 0x8e516a4357d217405313e40b18357b82bb460917..
% Contract name: Unitroller
% Contract at 0x4dCf7407AE5C07f8681e1659f626E114A7667339
% Proxy name: Comptroller
% Proxy at: 0x48c5e896d241Afd1Aee73ae19259A2e234256A85


% No.4 0x41d5d79431a913c4ae7d69a668ecdfe5ff9dfb68
% ABI for the implementation contract at 0xbafe01ff935c7305907c33bf824352ee5979b526, likely using a custom proxy implementation.
% Previously recorded to be on 0x234b619b4f4e405665f7d94f2ce60c24256032b5.
% Contract name: INV
% Contract at 0x41d5d79431a913c4ae7d69a668ecdfe5ff9dfb68
% Proxy name: N/A
% Proxy at: N/A


% No.5 0x010caccf546de952c5591b7018340549be2eb641
% ABI for the implementation contract at 0xbafe01ff935c7305907c33bf824352ee5979b526, likely using a custom proxy implementation.
% Previously recorded to be on 0x234b619b4f4e405665f7d94f2ce60c24256032b5.
% Contract name: Pool
% Contract at 0x010caccf546de952c5591b7018340549be2eb641
% Proxy name: N/A
% Proxy at: N/A



% The more complete list of the top addresses called by Compound DAOs are given below (contracts that are called more than 30 times):

% ('0x3d9819210a31b4961b30ef54be2aed79b9c9cd3b', 202),
%  ('0x0831172b9b136813b0b35e7cc898b1398bb4d7e7', 152),
%  ('0x4dcf7407ae5c07f8681e1659f626e114a7667339', 149),
%  ('0x41d5d79431a913c4ae7d69a668ecdfe5ff9dfb68', 83),
%  ('0x010caccf546de952c5591b7018340549be2eb641', 83),
%  ('0x0cec1a9154ff802e7934fc916ed7ca50bde6844e', 66),
%  ('0xe2e17b2cbbf48211fa7eb8a875360e5e39ba2602', 49),
%  ('0x7740792812a00510b50022d84e5c4ac390e01417', 48),
%  ('0x97990b693835da58a281636296d2bf02787dea17', 46),
%  ('0xde30da39c46104798bb5aa3fe8b9e0e1f348163f', 40),
%  ('0x865377367054516e17014ccded1e7d814edc9ce4', 40),
%  ('0xe8929afd47064efd36a7fb51da3f8c5eb40c4cb4', 38),
%  ('0x8d5ed43dca8c2f7dfb20cf7b53cc7e593635d7b9', 34)



% Here are the actual addresses that we processed:


% 0x010caccf546de952c5591b7018340549be2eb641
% 0x010caccf546de952c5591b7018340549be2eb641.zip
% 0x41d5d79431a913c4ae7d69a668ecdfe5ff9dfb68
% 0x41d5d79431a913c4ae7d69a668ecdfe5ff9dfb68.zip
% 0x48c5e896d241afd1aee73ae19259a2e234256a85
% 0x48c5e896d241afd1aee73ae19259a2e234256a85.zip
% 0xbafe01ff935c7305907c33bf824352ee5979b526
% 0xbafe01ff935c7305907c33bf824352ee5979b526.zip
% 0xc6df585f8721bfafbb1580bd4034315696eab9ca
% 0xc6df585f8721bfafbb1580bd4034315696eab9ca.zip


% Also explain the motivation of why choose the Compound DAO 
% 1. very popular and mature, well tested, less chance to be buggy
% 2. DAOs are gaining more popularity (explain DAO here as well)
